In [5]:
from mt5_connector import mt5_connector
import MetaTrader5 as mt5
from teleInformer import TeleInformer
import dataPreprocesser as dp

import pandas as pd


admiral_demo = mt5_connector(server='AdmiralMarkets-Demo',
                             login=40927195,
                             password='4740lpQbRYg7',
                             path="C:/metatraders/admiral/terminal64.exe",
                             timedelta=3)

Informer = TeleInformer(-414687545, '1155052470:AAELrvfzH_h2zMucZ7BTTaKeKS5lI01yrFg')

AdmiralMarkets-Demo initialize() success


In [21]:
import imp

In [22]:
imp.reload(mt5)

<module 'MetaTrader5' from 'C:\\Anaconda3\\lib\\site-packages\\MetaTrader5\\__init__.py'>

In [6]:
valid_symbols = ['GBPJPY', 'GBPUSD', "USDJPY", "USDCAD", "EURUSD"]

In [7]:
import os
import pickle 

class Strategy:
    
    def __init__(self, asset):
        self.asset = asset
        
        up_path = os.path.join(asset, "up", "model", "model.sav")
        down_path = os.path.join(asset, "down", "model", "model.sav")
        
        self.up_predictor = pickle.load(open(up_path, 'rb'))
        self.down_predictor = pickle.load(open(down_path, 'rb'))


In [8]:
s_GBPUSD = Strategy("GBPUSD")
s_GBPJPY = Strategy("GBPJPY")
s_USDJPY = Strategy("USDJPY")
s_USDCAD = Strategy("USDCAD")
s_EURUSD = Strategy("EURUSD")

In [9]:
from datetime import datetime, timedelta
buffer_size = timedelta(hours=150)
end_date = datetime.now().replace(microsecond=0, second=0, minute=0)
start_date = end_date - buffer_size

strategies = [s_GBPUSD, s_GBPJPY, s_USDJPY, s_USDCAD, s_EURUSD]

In [24]:
print(start_date)
print(end_date)
    
while True:
    

    if datetime.now().minute % 15 == 0:
        end_date = datetime.now().replace(microsecond=0, second=0)
        if currentTime != end_date:
            currentTime = end_date
            start_date = end_date - buffer_size
            print(start_date)
            print(end_date)

2020-08-24 16:30:00
2020-08-30 22:30:00
2020-08-24 16:35:00
2020-08-30 22:35:00
2020-08-24 16:40:00
2020-08-30 22:40:00
2020-08-24 16:45:00
2020-08-30 22:45:00
2020-08-24 16:50:00
2020-08-30 22:50:00


KeyboardInterrupt: 

In [ ]:

buffer_size = timedelta(hours=200)
currentTime = 0
while True:

    if datetime.now().minute % 15 == 0:
        end_date = datetime.now().replace(microsecond=0, second=0)
        if currentTime != end_date:
            currentTime = end_date
            start_date = end_date - buffer_size
        
        priceset = pd.DataFrame() 
        while priceset.empty:
            index_df = pd.DataFrame(index=pd.date_range(start_date, end_date, freq="15min"))
            index_df["hour"] = index_df.index.hour
            index_df["minute"] = index_df.index.minute
            print(start_date)
            print(end_date)
            priceset = dp.get_timesnp(admiral_demo, 
                           valid_symbols, 
                           start_date,  
                           end_date, 
                           index_df, 
                           timeframe=mt5.TIMEFRAME_M15, 
                           pricetypes=["high", "low", "close", "tick_volume"])

            priceset = priceset.fillna(method='ffill')
            for each_sym in valid_symbols:
                high_hour = dp.extract_max(priceset, each_sym + "_high", 4, new_column=None)
                low_hour = dp.extract_min(priceset, each_sym + "_low", 4, new_column=None)
                priceset = priceset.join(high_hour).join(low_hour)
            priceset = priceset.dropna(axis=0)

            priceset =  dp.extract_windowMean(priceset, valid_symbols, column_postfixes=["_high", "_close", "_low"], periods=[4,12,24,48,100])
            priceset =  dp.extract_windowStdev(priceset, valid_symbols, column_postfixes=["_high", "_close", "_low"], periods=[4,12,24,48,100])
            priceset =  dp.extract_windowCumulativeSum(priceset, valid_symbols, column_postfixes=["_high", "_close", "_low"], periods=[4,12,24,48,100])
            priceset =  dp.extract_candleDesc(priceset, valid_symbols)
            priceset =  dp.extract_meanDistance(priceset, valid_symbols, column_postfixes=["_high", "_close", "_low"], periods=[4,12,24,48,100])
            priceset =  dp.extract_maxDistance(priceset, valid_symbols, column_postfixes=["_high", "_close", "_low"], periods=[4,12,24,48,100])
            priceset =  dp.extract_minDistance(priceset, valid_symbols, column_postfixes=["_high", "_close", "_low"], periods=[4,12,24,48,100])
            priceset = priceset.dropna(axis=0)
#         features = list(filter(lambda x: name in x, priceset.columns)) 
#         priceset = priceset[features]
    
        for each_col in priceset.columns:
            priceset["ar_0_"+each_col] = (priceset[each_col] - priceset[each_col].shift(1))
            priceset["ar_1_"+each_col] = (priceset[each_col] - priceset[each_col].shift(2))
            priceset["ar_2_"+each_col] = (priceset[each_col] - priceset[each_col].shift(3))
            priceset["ar_3_"+each_col] = (priceset[each_col] - priceset[each_col].shift(4))
            priceset["ar_4_"+each_col] = (priceset[each_col] - priceset[each_col].shift(5))
            
        priceset = priceset.dropna(axis=0)
        
        for each_strategy in strategies:
            asset = each_strategy.asset
            tick = admiral_demo.get_tick(asset)
            spread = tick.ask - tick.bid
            candle_min = priceset["min_"+asset + "_low_4"][-1]
            candle_max = priceset["max_"+asset + "_high_4"][-1]
            features = list(filter(lambda x: asset in x, priceset.columns)) 
            X = priceset[features]
            features_to_learn = list(filter(lambda x: "ar" in x, X.columns))
            X = priceset[features_to_learn].join(index_df[["hour", "minute"]])
            X = pd.DataFrame(X.iloc[-2, :]).T
            print(asset)
            up_signal = each_strategy.up_predictor.predict(X)[0]
            down_signal = each_strategy.down_predictor.predict(X)[0]
            print(up_signal, down_signal)
            
            if up_signal == 1 and down_signal == 0:
                signal = 0      
            elif down_signal == 1 and up_signal == 0:
                signal = 1
            else:
                signal = -1
            
            positions = admiral_demo.get_positions(asset)
            position_cnt = 0
            if len(positions) > 0:
                for each_position in positions: 
                    if each_position.magic == 15777:
                        position_cnt = position_cnt + 1
                        position_direction = each_position.type
                        if signal != position_direction:
                            result = admiral_demo.close_position(each_position, filltype=0)
                            Informer.send_mess(str(result))
                        else:
                            if position_direction == 0:
                                result = admiral_demo.modify_tpsl(each_position, sl=candle_min)
                                Informer.send_mess(str(result))                            
                            else:
                                result = admiral_demo.modify_tpsl(each_position, sl=candle_max+spread)
                                Informer.send_mess(str(result))   
                        
            positions = admiral_demo.get_positions(asset)

            if signal == 0 and position_cnt == 0:
                print("Buy", asset)
                result = admiral_demo.open_position("BUY", asset, filltype=0, lot=0.01, comment="15Min", magic=15777)
                Informer.send_mess(str(result))
            if signal == 1 and position_cnt == 0:
                print("Sell", asset) 
                result = admiral_demo.open_position("SELL", asset, filltype=0, lot=0.01, comment="15Min", magic=15777)
                Informer.send_mess(str(result))


2020-08-23 00:30:00
2020-08-31 08:30:00


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 178.68it/s]


GBPUSD
0 0
GBPJPY
0 1
USDJPY
0 1
Sell USDJPY
USDCAD
1 0
EURUSD
0 0
2020-08-23 00:30:00
2020-08-31 08:30:00


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 142.94it/s]


GBPUSD
0 0
GBPJPY
0 1
USDJPY
0 1
USDCAD
1 0
EURUSD
0 0
2020-08-23 00:30:00
2020-08-31 08:30:00


...

GBPUSD
0 0
GBPJPY
0 0
USDJPY
0 0
USDCAD
0 1
EURUSD
0 0
2020-08-23 11:45:00
2020-08-31 19:45:00


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 172.51it/s]


GBPUSD
0 0
GBPJPY
0 0
USDJPY
0 0
USDCAD
0 1
EURUSD
0 0
2020-08-23 11:45:00
2020-08-31 19:45:00


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 185.29it/s]


GBPUSD
0 0
GBPJPY
0 0
USDJPY
0 0
USDCAD
0 1
EURUSD
0 0


In [8]:
admiral_demo.open_position("BUY", "NZDCAD", filltype=0, lot=0.01, comment="15Min")

OrderSendResult(retcode=10009, deal=0, order=1054747591, volume=0.01, price=0.0, bid=0.0, ask=0.0, comment='15Min', request_id=23, retcode_external=0, request=TradeRequest(action=1, magic=4577, order=0, symbol='NZDCAD', volume=0.01, price=0.88019, stoplimit=0.0, sl=0.0, tp=0.0, deviation=1, type=0, type_filling=0, type_time=0, expiration=0, comment='15Min', position=0, position_by=0))

In [44]:
X["hour"]

KeyError: 'hour'

In [43]:
for i in X.columns:
    print(i)

ar_0_GBPUSD_high
ar_1_GBPUSD_high
ar_2_GBPUSD_high
ar_3_GBPUSD_high
ar_4_GBPUSD_high
ar_0_GBPUSD_low
ar_1_GBPUSD_low
ar_2_GBPUSD_low
ar_3_GBPUSD_low
ar_4_GBPUSD_low
ar_0_GBPUSD_close
ar_1_GBPUSD_close
ar_2_GBPUSD_close
ar_3_GBPUSD_close
ar_4_GBPUSD_close
ar_0_GBPUSD_tick_volume
ar_1_GBPUSD_tick_volume
ar_2_GBPUSD_tick_volume
ar_3_GBPUSD_tick_volume
ar_4_GBPUSD_tick_volume
ar_0_max_GBPUSD_high_4
ar_1_max_GBPUSD_high_4
ar_2_max_GBPUSD_high_4
ar_3_max_GBPUSD_high_4
ar_4_max_GBPUSD_high_4
ar_0_min_GBPUSD_low_4
ar_1_min_GBPUSD_low_4
ar_2_min_GBPUSD_low_4
ar_3_min_GBPUSD_low_4
ar_4_min_GBPUSD_low_4
ar_0_mean_GBPUSD_high_4
ar_1_mean_GBPUSD_high_4
ar_2_mean_GBPUSD_high_4
ar_3_mean_GBPUSD_high_4
ar_4_mean_GBPUSD_high_4
ar_0_mean_GBPUSD_high_12
ar_1_mean_GBPUSD_high_12
ar_2_mean_GBPUSD_high_12
ar_3_mean_GBPUSD_high_12
ar_4_mean_GBPUSD_high_12
ar_0_mean_GBPUSD_high_24
ar_1_mean_GBPUSD_high_24
ar_2_mean_GBPUSD_high_24
ar_3_mean_GBPUSD_high_24
ar_4_mean_GBPUSD_high_24
ar_0_mean_GBPUSD_high_48
ar_1_

Tick(time=1598647942, bid=1.31119, ask=1.31134, last=0.0, volume=0, time_msc=1598647942606, flags=6, volume_real=0.0)

In [28]:
positions = admiral_demo.get_positions("USDCAD")
positions

(TradePosition(ticket=1054721509, time=1598647246, time_msc=1598647246092, time_update=1598647246, time_update_msc=1598647246092, type=1, magic=4577, identifier=1054721509, reason=3, volume=0.01, price_open=1.31113, sl=0.0, tp=0.0, price_current=1.31163, swap=0.0, profit=-0.38, symbol='USDCAD', comment='', external_id=''),)

In [30]:
result = admiral_demo.modify_tpsl(positions[0], sl=1.4)
result

OrderSendResult(retcode=10009, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Request executed', request_id=57, retcode_external=0, request=TradeRequest(action=6, magic=0, order=0, symbol='', volume=0.0, price=0.0, stoplimit=0.0, sl=1.4, tp=0.0, deviation=0, type=0, type_filling=0, type_time=0, expiration=0, comment='', position=1054721509, position_by=0))

In [26]:
result

In [27]:
i = 0
while i!=5:
    i = i +1
    
    

In [37]:
priceset = dp.get_timesnp(admiral_demo, 
                           valid_symbols, 
                           start_date,  
                           end_date, 
                           index_df, 
                           timeframe=mt5.TIMEFRAME_H1, 
                           pricetypes=["high", "low", "close", "tick_volume"])

priceset = priceset.fillna(method='ffill')
priceset = priceset.dropna(axis=0)

priceset =  dp.extract_windowMean(priceset, valid_symbols, column_postfixes=["_high", "_close", "_low"], periods=[6,12,24,120])
#priceset = priceset.dropna(axis=0)
priceset =  dp.extract_windowStdev(priceset, valid_symbols, column_postfixes=["_high", "_close", "_low"], periods=[6,12,24,120])
priceset =  dp.extract_windowCumulativeSum(priceset, valid_symbols, column_postfixes=["_high", "_close", "_low"], periods=[6,12,24,120])
priceset =  dp.extract_candleDesc(priceset, valid_symbols)
priceset =  dp.extract_meanDistance(priceset, valid_symbols, column_postfixes=["_high", "_close", "_low"], periods=[6,12,24,120])
priceset =  dp.extract_maxDistance(priceset, valid_symbols, column_postfixes=["_high", "_close", "_low"], periods=[6,12,24,120])
priceset =  dp.extract_minDistance(priceset, valid_symbols, column_postfixes=["_high", "_close", "_low"], periods=[6,12,24,120])


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 121.99it/s]


In [41]:
priceset

,hour,GBPJPY_high,GBPJPY_low,GBPJPY_close,GBPJPY_tick_volume,GBPUSD_high,GBPUSD_low,GBPUSD_close,GBPUSD_tick_volume,USDJPY_high,...,mindist_EURUSD_high_24,mindist_EURUSD_high_120,mindist_EURUSD_close_6,mindist_EURUSD_close_12,mindist_EURUSD_close_24,mindist_EURUSD_close_120,mindist_EURUSD_low_6,mindist_EURUSD_low_12,mindist_EURUSD_low_24,mindist_EURUSD_low_120
2020-08-24 00:00:00,0,138.527,138.361,138.383,239.0,1.30942,1.30835,1.30881,153.0,105.801,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-24 01:00:00,1,138.705,138.377,138.682,1409.0,1.30990,1.30876,1.30982,987.0,105.920,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-24 02:00:00,2,138.691,138.602,138.609,1892.0,1.31012,1.30907,1.30922,884.0,105.887,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-24 03:00:00,3,138.678,138.412,138.422,3541.0,1.30959,1.30859,1.30871,1542.0,105.938,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-24 04:00:00,4,138.494,138.311,138.477,3536.0,1.30971,1.30822,1.30942,1993.0,105.792,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-24 05:00:00,5,138.508,138.426,138.462,2151.0,1.30974,1.30840,1.30861,1540.0,105.831,...,NaN,NaN,0.00000,NaN,NaN,NaN,0.00003,NaN,NaN,NaN
2020-08-24 06:00:00,6,138.516,138.453,138.458,2221.0,1.30915,1.30837,1.30840,1512.0,105.845,...,NaN,NaN,0.00000,NaN,NaN,NaN,0.00000,NaN,NaN,NaN
2020-08-24 07:00:00,7,138.573,138.454,138.530,2416.0,1.30920,1.30834,1.30874,1586.0,105.875,...,NaN,NaN,0.00066,NaN,NaN,NaN,0.00025,NaN,NaN,NaN
2020-08-24 08:00:00,8,138.661,138.530,138.647,2535.0,1.31033,1.30854,1.31026,1647.0,105.920,...,NaN,NaN,0.00160,NaN,NaN,NaN,0.00063,NaN,NaN,NaN
2020-08-24 09:00:00,9,138.737,138.565,138.696,3626.0,1.31109,1.30940,1.31067,3276.0,105.871,...,NaN,NaN,0.00182,NaN,NaN,NaN,0.00099,NaN,NaN,NaN


In [13]:
a = None
a = 1
if a:
    print("a is None?")

a is None?


In [14]:
positions = admiral_demo.get_positions("USDCHF")

In [19]:
positions[0].type

1

In [ ]:
def resultToStr(result):
    result_string = result.

In [186]:
" ".join(["i", "love", "eating"])

'i love eating'

"OrderSendResult(retcode=10009, deal=0, order=1054247938, volume=0.01, price=0.0, bid=0.0, ask=0.0, comment='Request executed', request_id=115, retcode_external=0, request=TradeRequest(action=1, magic=4577, order=0, symbol='GBPUSD', volume=0.01, price=1.31078, stoplimit=0.0, sl=0.0, tp=0.0, deviation=1, type=0, type_filling=0, type_time=0, expiration=0, comment='', position=0, position_by=0))"

In [181]:
each_position

TradePosition(ticket=1054247012, time=1598338807, time_msc=1598338807702, time_update=1598338807, time_update_msc=1598338807702, type=1, magic=4577, identifier=1054247012, reason=3, volume=0.01, price_open=1.18075, sl=0.0, tp=0.0, price_current=1.18118, swap=0.0, profit=-0.43, symbol='EURUSD', comment='', external_id='')

In [162]:
admiral_demo.open_position("SELL", asset, filltype=0, lot=0.01)

OrderSendResult(retcode=10009, deal=0, order=1054235458, volume=0.01, price=0.0, bid=0.0, ask=0.0, comment='Request executed', request_id=3, retcode_external=0, request=TradeRequest(action=1, magic=4577, order=0, symbol='EURUSD', volume=0.01, price=1.17869, stoplimit=0.0, sl=0.0, tp=0.0, deviation=1, type=1, type_filling=0, type_time=0, expiration=0, comment='', position=0, position_by=0))

In [150]:
pd.DataFrame(X.iloc[-2, :]).T

,ar_0_EURUSD_high,ar_1_EURUSD_high,ar_0_EURUSD_low,ar_1_EURUSD_low,ar_0_EURUSD_close,ar_1_EURUSD_close,ar_0_EURUSD_tick_volume,ar_1_EURUSD_tick_volume,ar_0_mean_EURUSD_high_6,ar_1_mean_EURUSD_high_6,...,ar_0_mindist_EURUSD_close_120,ar_1_mindist_EURUSD_close_120,ar_0_mindist_EURUSD_low_6,ar_1_mindist_EURUSD_low_6,ar_0_mindist_EURUSD_low_12,ar_1_mindist_EURUSD_low_12,ar_0_mindist_EURUSD_low_24,ar_1_mindist_EURUSD_low_24,ar_0_mindist_EURUSD_low_120,ar_1_mindist_EURUSD_low_120
2020-08-24 22:00:00,-0.00065,-0.00127,-0.00074,-0.00091,-0.00002,-0.00058,16.0,-589.0,-0.000868,-0.001707,...,-0.00002,-0.00058,0.0,-0.00017,0.0,-0.00017,-0.00069,-0.00086,-0.00074,-0.00091


In [115]:
enddate = datetime.now().replace(microsecond=0, second=0, minute=0) + timedelta(hours=3)

In [ ]:
index_df

In [116]:
priceset = dp.get_timesnp(admiral_demo, 
               ["GBPUSD"], 
               start_date,  
               enddate, 
               index_df, 
               timeframe=mt5.TIMEFRAME_H1, 
               pricetypes=["high", "low", "close", "tick_volume"])
priceset

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 111.17it/s]


,hour,GBPUSD_high,GBPUSD_low,GBPUSD_close,GBPUSD_tick_volume
2020-08-18 17:00:00,17,1.32377,1.32048,1.32176,7693.0
2020-08-18 18:00:00,18,1.32319,1.32098,1.32257,4276.0
2020-08-18 19:00:00,19,1.32362,1.32225,1.32341,3793.0
2020-08-18 20:00:00,20,1.32440,1.32320,1.32364,3800.0
2020-08-18 21:00:00,21,1.32439,1.32336,1.32436,2314.0
2020-08-18 22:00:00,22,1.32495,1.32376,1.32390,2498.0
2020-08-18 23:00:00,23,1.32441,1.32370,1.32378,1492.0
2020-08-19 00:00:00,0,1.32454,1.32285,1.32383,910.0
2020-08-19 01:00:00,1,1.32451,1.32377,1.32451,1340.0
2020-08-19 02:00:00,2,1.32543,1.32423,1.32523,1222.0


In [98]:
dp.extract_bars(admiral_demo, "GBPUSD", mt5.TIMEFRAME_H1, start_date, end_date)

,time,open,high,low,close,tick_volume,spread,real_volume
0,2020-08-18 14:00:00,1.31607,1.31783,1.31531,1.31773,4168,8,0
1,2020-08-18 15:00:00,1.31774,1.31991,1.31718,1.31944,4616,8,0
2,2020-08-18 16:00:00,1.31945,1.32402,1.31886,1.32286,6719,8,0
3,2020-08-18 17:00:00,1.32292,1.32377,1.32048,1.32176,7693,8,0
4,2020-08-18 18:00:00,1.32176,1.32319,1.32098,1.32257,4276,8,0
5,2020-08-18 19:00:00,1.32257,1.32362,1.32225,1.32341,3793,8,0
6,2020-08-18 20:00:00,1.32339,1.32440,1.32320,1.32364,3800,8,0
7,2020-08-18 21:00:00,1.32364,1.32439,1.32336,1.32436,2314,8,0
8,2020-08-18 22:00:00,1.32436,1.32495,1.32376,1.32390,2498,8,0
9,2020-08-18 23:00:00,1.32394,1.32441,1.32370,1.32378,1492,8,0


In [68]:
dp.get_timesnp(admiral_demo, 
                       valid_symbols, 
                       start_date,  
                       end_date, 
                       index_df, 
                       timeframe=mt5.TIMEFRAME_H1, 
                       pricetypes=["high", "low", "close", "tick_volume"])

100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 117.72it/s]


,hour,GBPJPY_high,GBPJPY_low,GBPJPY_close,GBPJPY_tick_volume,GBPUSD_high,GBPUSD_low,GBPUSD_close,GBPUSD_tick_volume
2020-08-16 13:53:00,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-16 14:53:00,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-16 15:53:00,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-16 16:53:00,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-16 17:53:00,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-16 18:53:00,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-16 19:53:00,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-16 20:53:00,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-16 21:53:00,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-16 22:53:00,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [107]:
enddate = datetime.now().replace(microsecond=0, second=0, minute=0) + timedelta(hours=10)
dp.extract_bars(admiral_demo, "GBPJPY", mt5.TIMEFRAME_H1, start_date, enddate)


,time,open,high,low,close,tick_volume,spread,real_volume
0,2020-08-18 14:00:00,138.908,139.032,138.825,138.966,4175,19,0
1,2020-08-18 15:00:00,138.966,139.187,138.867,139.166,5052,19,0
2,2020-08-18 16:00:00,139.161,139.433,139.091,139.333,7729,19,0
3,2020-08-18 17:00:00,139.334,139.495,139.307,139.380,9632,19,0
4,2020-08-18 18:00:00,139.380,139.507,139.250,139.419,4972,19,0
5,2020-08-18 19:00:00,139.419,139.513,139.359,139.400,3843,19,0
6,2020-08-18 20:00:00,139.406,139.575,139.405,139.527,3003,19,0
7,2020-08-18 21:00:00,139.528,139.632,139.524,139.591,2048,19,0
8,2020-08-18 22:00:00,139.591,139.608,139.507,139.507,2324,19,0
9,2020-08-18 23:00:00,139.516,139.588,139.501,139.531,1481,19,0


In [52]:


buffer = {}
strategies = [Strategy]

priceset = dp.get_timesnp(admiral_demo, 
                       valid_symbols, 
                       start_date,  
                       end_date, 
                       index_df, 
                       timeframe=mt5.TIMEFRAME_H1, 
                       pricetypes=["high", "low", "close", "tick_volume"])
    
    
    

NameError: name 'index_df' is not defined